In [2]:
import csv
import pandas as pd
from data.SpotifyDataset import SpotifyDataset

In [62]:
import lightgbm as lgb
import numpy as np

In [9]:
train_file = 'data/train_data.csv.zip'
track_file = 'data/track_feats.csv.zip'
train = pd.read_csv(train_file)
track_feats = pd.read_csv(track_file)

In [10]:
train

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,hist_user_behavior_reason_start_trackdone,hist_user_behavior_reason_start_trackerror,hist_user_behavior_reason_end_appload,hist_user_behavior_reason_end_backbtn,hist_user_behavior_reason_end_clickrow,hist_user_behavior_reason_end_endplay,hist_user_behavior_reason_end_fwdbtn,hist_user_behavior_reason_end_logout,hist_user_behavior_reason_end_remote,hist_user_behavior_reason_end_trackdone
0,63_0000075d-50c4-4c13-b7d1-e91fdbce83bc,1,20,t_1a73002a-d065-4a33-b98a-a3b0443310be,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,63_0000075d-50c4-4c13-b7d1-e91fdbce83bc,2,20,t_1a73002a-d065-4a33-b98a-a3b0443310be,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,63_0000075d-50c4-4c13-b7d1-e91fdbce83bc,3,20,t_16bc2816-8daf-449a-a0f7-93ee884ffdda,0,1,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,63_0000075d-50c4-4c13-b7d1-e91fdbce83bc,4,20,t_71d34c2f-1c87-43df-83ba-00832e12194c,0,1,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,63_0000075d-50c4-4c13-b7d1-e91fdbce83bc,5,20,t_a66ea088-b357-449a-8a1e-64dd0b8d6cb5,0,1,1,0,1,1,...,0,0,0,0,0,1,0,0,0,0
5,63_0000075d-50c4-4c13-b7d1-e91fdbce83bc,6,20,t_a66ea088-b357-449a-8a1e-64dd0b8d6cb5,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6,63_0000075d-50c4-4c13-b7d1-e91fdbce83bc,7,20,t_2a55e695-7894-42aa-b920-52d1cbaf164c,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
7,63_0000075d-50c4-4c13-b7d1-e91fdbce83bc,8,20,t_a004df37-97e0-4c20-999e-2844ddf9e733,0,0,0,1,0,1,...,1,0,0,0,0,0,0,0,0,1
8,63_0000075d-50c4-4c13-b7d1-e91fdbce83bc,9,20,t_b362a386-e30e-46be-bb44-af6efd02260f,0,1,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
9,63_0000075d-50c4-4c13-b7d1-e91fdbce83bc,10,20,t_508ab67d-dd11-478c-8b45-5c522df5e03d,1,1,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [11]:
# Pre-process
# how to do this for validation (with track_file?)
SpotifyDataset = SpotifyDataset(train_file, track_file)

In [15]:
random_num = 2
print(SpotifyDataset.__getitem__(random_num)[0].columns)

Index(['session_position', 'session_length', 'skip_1', 'skip_2', 'skip_3',
       'not_skipped', 'context_switch', 'no_pause_before_play',
       'short_pause_before_play', 'long_pause_before_play',
       'hist_user_behavior_n_seekfwd', 'hist_user_behavior_n_seekback',
       'hist_user_behavior_is_shuffle', 'hour_of_day', 'premium',
       'context_type_catalog', 'context_type_charts',
       'context_type_editorial_playlist', 'context_type_personalized_playlist',
       'context_type_radio', 'context_type_user_collection',
       'hist_user_behavior_reason_start_appload',
       'hist_user_behavior_reason_start_backbtn',
       'hist_user_behavior_reason_start_clickrow',
       'hist_user_behavior_reason_start_endplay',
       'hist_user_behavior_reason_start_fwdbtn',
       'hist_user_behavior_reason_start_playbtn',
       'hist_user_behavior_reason_start_remote',
       'hist_user_behavior_reason_start_trackdone',
       'hist_user_behavior_reason_start_trackerror',
       'hist_u

In [36]:
X = SpotifyDataset.__getitem__(0)[0]
print(X)

   session_position  session_length  skip_1  skip_2  skip_3  not_skipped  \
0                 1              20       1       1       1            0   
1                 2              20       0       0       0            0   
2                 3              20       0       1       1            0   
3                 4              20       0       1       1            0   
4                 5              20       0       1       1            0   
5                 6              20       0       0       1            0   
6                 7              20       0       0       0            1   
7                 8              20       0       0       0            1   
8                 9              20       0       1       1            0   
9                10              20       1       1       1            0   

   context_switch  no_pause_before_play  short_pause_before_play  \
0               0                     0                        0   
1               0          

In [59]:
## extract X train for 1 example
# TODO: this and y train is not padded
# need to add more examples
# do i have to call encode
X_train = X.drop(['skip_2', 'mode'], axis=1)
print(X_train)

   session_position  session_length  skip_1  skip_3  not_skipped  \
0                 1              20       1       1            0   
1                 2              20       0       0            0   
2                 3              20       0       1            0   
3                 4              20       0       1            0   
4                 5              20       0       1            0   
5                 6              20       0       1            0   
6                 7              20       0       0            1   
7                 8              20       0       0            1   
8                 9              20       0       1            0   
9                10              20       1       1            0   

   context_switch  no_pause_before_play  short_pause_before_play  \
0               0                     0                        0   
1               0                     0                        0   
2               0                     0        

In [60]:
## extract y train for 1 example
y_train = X.loc[:, 'skip_2']
print(y_train)

0    1
1    0
2    1
3    1
4    1
5    0
6    0
7    0
8    1
9    1
Name: skip_2, dtype: int64


In [68]:
val_file = 'data/val_data.csv'
val = pd.read_csv(val_file)
# val
# Temp for testing
x_val_example = SpotifyDataset.__getitem__(4)[0]
X_val = x_val_example.drop(['skip_2', 'mode'], axis=1)
y_val = x_val_example.loc[:, 'skip_2']

0    0.0
1    0.0
2    0.0
3    1.0
4    0.0
5    0.0
6    0.0
7    1.0
8    0.0
9    0.0
Name: skip_2, dtype: float64


In [74]:
def evaluate_model(const_preds, y_truth_list):
    tmp_preds_constant_mdl = pd.DataFrame({'pred': const_preds, 'len': y_truth_list.apply(len)})
    preds_lists_stp = tmp_preds_constant_mdl.apply(lambda x: [x['pred']]*x['len'], axis=1)
    del tmp_preds_constant_mdl
    return evaluate(preds_lists_stp.tolist(), y_truth_list.tolist())

In [78]:
def learning_rate_decay_power_0995(current_iter):
    base_learning_rate = 0.15
    lr = base_learning_rate  * np.power(.998, current_iter)
    return lr if lr > 1e-2 else 1e-2


mdl = lgb.LGBMClassifier(max_depth=-1, min_child_samples=400, 
              random_state=314, silent=True, metric='None', 
              n_jobs=4, n_estimators=500, learning_rate=0.1,
              **{'colsample_bytree': 0.75, 'min_child_weight': 1, 
               'num_leaves': 60, 'subsample': 0.75}
             )

n_fit = None

# X = pd.concat([X_trn, X_trk[0]], axis=1)
fit_params = {'eval_names': ['train', 'early_stop'],
              'eval_set': [(X_val, y_val), 
                           (X_val, y_val)],
              'eval_metric': 'binary_error',
              'verbose':50, 'early_stopping_rounds':60,
              'callbacks':[lgb.reset_parameter(learning_rate=learning_rate_decay_power_0995)]}

mdl.fit(X_train, y_train, **fit_params)

prob_pred = mdl.predict_proba(X_train) #.loc[id_stp,:])[:,1]
print(prob_pred)
evaluate_model(prob_pred>0.55, y_val)

Training until validation scores don't improve for 60 rounds
[50]	train's binary_error: 0.8	early_stop's binary_error: 0.8
Early stopping, best iteration is:
[1]	train's binary_error: 0.8	early_stop's binary_error: 0.8
[[0.4 0.6]
 [0.4 0.6]
 [0.4 0.6]
 [0.4 0.6]
 [0.4 0.6]
 [0.4 0.6]
 [0.4 0.6]
 [0.4 0.6]
 [0.4 0.6]
 [0.4 0.6]]


TypeError: object of type 'float' has no len()